-Merging DataFrames

-Pandas Idioms

-Groupby

-Scales

-Pivot Table

-Date/Time Functionality

# Merging Dataframes

Nessa seção vamos ver como juntar dataframes, seja juntando eles horizontalmente(merge) ou verticalmente(concate).

Antes de prosseguirmos, será útil revisar algumas definições de teoria de grupos.

![Venn Diagram](merging1.png)

Isso é um Diagrama de Venn.
O círculo da esquerda representa os estudantes de uma Universidade.
O círculo da direita representa os funcionários da Universidade.
A região de superposição representa aqueles que são estudantes e funcionários da Universidade.


No caso do Pandas, podemos pensar no caso em que podemos ter essas duas populações como índices em DataFrames separados, talvez o Nome da pessoa, por exemplo.

Quando queremos juntar esses DataFrames, nós temos algumas escolhas a fazer.

Primeiro, se quisermos essas pessoas independente se elas são estudantes ou funcionários.Então queremos fazer uma União em teoria de grupos, ou uma 'full outer join' na nomenclatura de dataBase.

![Union](merging2.png)

Se quisermos aqueles que são estudantes E funcionários.
Então queremos fazer uma Intersecção em teoria de grupos, ou uma 'inner join' na nomenclatura de dataBase.

![Intersection](merging3.png)

In [82]:
# Vamos ver um exemplo

import pandas as pd

#Primeiro vamos criar os DataFrames 'students' e 'staff'
staff_df=pd.DataFrame([{'Name':'Kelly','Role':'Director of HR'},
                      {'Name':'Sally','Role':'Course liasion'},
                      {'Name':'James','Role':'Grader'}])

student_df=pd.DataFrame([{'Name':'James','School':'Business'},
                         {'Name':'Mike','School':'Law'},
                          {'Name':'Sally','School':'Engineering'}])
print(staff_df)
print(student_df)

    Name            Role
0  Kelly  Director of HR
1  Sally  Course liasion
2  James          Grader
    Name       School
0  James     Business
1   Mike          Law
2  Sally  Engineering


# Merge

In [83]:
# IMPORTANTE: Os DataFrames devem ser indexados na coluna que queremos juntar, como queremos juntar os dataFrames por nome,
#vamos setar a coluna 'Name' como indice em ambos os Dataframes.
staff_df.set_index('Name',inplace=True)
student_df.set_index('Name',inplace=True)

In [84]:
# Podemos ver que a Kelly é só estudante, o Mike é só funcionário e o James e a Sally são estudantes e funcionários.

### Union / outer

In [85]:
# Se quisermos unir esses DataFrames, nós chamamos a função merge() passando como parâmetros os DataFrames que queremos juntar,
# e modo(nessa caso 'outer'). Queremos usar os indices esquerdo e direito como colunas de 'junção'.

pd.merge(staff_df, student_df, how='outer', left_index=True, right_index=True)

,Role,School
Name,,
James,Grader,Business
Kelly,Director of HR,NaN
Mike,NaN,Law
Sally,Course liasion,Engineering


In [86]:
# Podemos ver que todos os nomes estão sendo listados no DataFrame resultante.
# E como Mike não tem um emprego na Universidade e Kelly não estuda na Universidade, as células são listadas como 'missing values'.

### Intersection / inner

In [87]:
# Se quisermos pegar a intersecção, ou seja, aqueles que são funcionários e estudam na Universidade, utilizamos o atributo 'inner'.
pd.merge(staff_df,student_df,how='inner', left_index=True, right_index=True)

,Role,School
Name,,
Sally,Course liasion,Engineering
James,Grader,Business


In [88]:
# Podemos ver que o resultado só tem aqueles que trabalham e estudam na Universidade.

### Set Addition

Existe outros dois casos quando queremos juntar DataFrames.

-Podemos querer pegar todas as pessoas que são funcionários, independentes se são estudantes ou não.
Se forem estudantes, vamos querer pegar essas informaçoes também. É o que é chamado de **left join**. O DataFrame da esquerda será preservado e o DataFrame da direita se juntará a ele.

-Podemos querer pegar todas as pessoas que são estudantes, independentes se são funcionários ou não.
Se forem funcionários, vamos querer pegar essas informaçoes também. É o que é chamado de **right join**. O DataFrame da direita será preservado e o DataFrame da esquerda se juntará a ele.

In [89]:
pd.merge(staff_df, student_df, how='left', left_index=True, right_index=True)

,Role,School
Name,,
Kelly,Director of HR,NaN
Sally,Course liasion,Engineering
James,Grader,Business


In [90]:
pd.merge(staff_df, student_df, how='right', left_index=True, right_index=True)

,Role,School
Name,,
James,Grader,Business
Mike,NaN,Law
Sally,Course liasion,Engineering


### 'on' Parameter

In [91]:
# Podemos fazer de outro jeito, não precisamos usar os parâmetros 'left_index' e 'right_index' indicando que queremos juntas as colunas.
# Podemos utilizar o parâmetro 'on' indicando a coluna que queremos mesclar.

In [92]:
# Nesse caso não precisamos utilizar a coluna 'Name' como índice.Nesse caso a diferença entre usar e não usar será no index do
# DataFrame retornado.
staff_df = staff_df.reset_index()
student_df = student_df.reset_index()

In [93]:
pd.merge(staff_df, student_df, how='outer', on='Name')

,Name,Role,School
0,Kelly,Director of HR,NaN
1,Sally,Course liasion,Engineering
2,James,Grader,Business
3,Mike,NaN,Law


In [94]:
pd.merge(staff_df, student_df, how='inner', on='Name')

,Name,Role,School
0,Sally,Course liasion,Engineering
1,James,Grader,Business


In [95]:
pd.merge(staff_df, student_df, how='right', on='Name')

,Name,Role,School
0,James,Grader,Business
1,Mike,NaN,Law
2,Sally,Course liasion,Engineering


In [96]:
pd.merge(staff_df, student_df, how='left', on='Name')

,Name,Role,School
0,Kelly,Director of HR,NaN
1,Sally,Course liasion,Engineering
2,James,Grader,Business


### Case that there is conflicts between the DataFrames

In [97]:
staff_df = pd.DataFrame([{'Name': 'Kelly', 'Role': 'Director of HR', 
                          'Location': 'State Street'},
                         {'Name': 'Sally', 'Role': 'Course liasion', 
                          'Location': 'Washington Avenue'},
                         {'Name': 'James', 'Role': 'Grader', 
                          'Location': 'Washington Avenue'}])
student_df = pd.DataFrame([{'Name': 'James', 'School': 'Business', 
                            'Location': '1024 Billiard Avenue'},
                           {'Name': 'Mike', 'School': 'Law', 
                            'Location': 'Fraternity House #22'},
                           {'Name': 'Sally', 'School': 'Engineering', 
                            'Location': '512 Wilson Crescent'}])

In [98]:
# Ambos os DataFrames possuem as colunas 'Name' e 'Location'.
# Mas, podemos ver que a coluna Location é diferente para a mesma pessoa. Isso se deve pelo fato de que a coluna 'Location' no DataFrame
# 'staff_df' se refere ao local do trabalho, enquanto no DataFrame 'student_df' se refere ao local que o estudante mora.
# Acontece que a função merge() preserva essa informação mas adiciona '_x' (representando o DataFrame da esquerda) e '_y'(representando o DataFrame da direita)
# à coluna em conflito.

# Exemplo, se quisermos as informações só dos funcionários
pd.merge(staff_df, student_df, how='left', on='Name')

# 'Location_x' representa a coluna 'Location' do DataFrame 'staff_d' e 'Location_y' representa a coluna 'Location' do Dataframe 'student_df'.

,Name,Role,Location_x,School,Location_y
0,Kelly,Director of HR,State Street,NaN,NaN
1,Sally,Course liasion,Washington Avenue,Engineering,512 Wilson Crescent
2,James,Grader,Washington Avenue,Business,1024 Billiard Avenue


### Multi-indexing and multiple columns

In [99]:
# Vamos supor que tenhamos dois DataFrames com as colunas 'First Name' e 'Last Name'.
# Pode ser que o primeiro nome dos funcionários e estudantes venham a coincidir, mas o último nome não.

# Nesse caso, utilizamos uma lista no parâmetro 'on' indicando múltiplas colunas para fazer a mesclagem.

# Exemplo:
staff_df = pd.DataFrame([{'First Name': 'Kelly', 'Last Name': 'Desjardins', 
                          'Role': 'Director of HR'},
                         {'First Name': 'Sally', 'Last Name': 'Brooks', 
                          'Role': 'Course liasion'},
                         {'First Name': 'James', 'Last Name': 'Wilde', 
                          'Role': 'Grader'}])
student_df = pd.DataFrame([{'First Name': 'James', 'Last Name': 'Hammond', 
                            'School': 'Business'},
                           {'First Name': 'Mike', 'Last Name': 'Smith', 
                            'School': 'Law'},
                           {'First Name': 'Sally', 'Last Name': 'Brooks', 
                            'School': 'Engineering'}])
staff_df

,First Name,Last Name,Role
0,Kelly,Desjardins,Director of HR
1,Sally,Brooks,Course liasion
2,James,Wilde,Grader


In [100]:
# Podemos ver que existe duas pessoas com o nome 'James', mas sobrenomes diferentes.
# Então, esperamos que a intersecção não pegue essas duas pessoas.
pd.merge(staff_df, student_df, how='inner', on=['First Name','Last Name'])

,First Name,Last Name,Role,School
0,Sally,Brooks,Course liasion,Engineering


# Concate

Juntar os DataFrames verticalmente.

In [101]:
# Exemplo: Vamos supor que tenhamos algum tipo de  informação ao longo de alguns anos e que cada arquivo tenha a informação apenas de um ano.

# Esses DataFrames estão bagunçados, então iremos evitar as mensages de aviso do jupyter utilizando '%%capture' e utilizar o
# parâmetro 'error_bad_lines' para ignorar as linhas que causam problema.

In [102]:
%%capture
df_2011 = pd.read_csv("datasets/college_scorecard/MERGED2011_12_PP.csv", error_bad_lines=False)
df_2012 = pd.read_csv("datasets/college_scorecard/MERGED2012_13_PP.csv", error_bad_lines=False)
df_2013 = pd.read_csv("datasets/college_scorecard/MERGED2013_14_PP.csv", error_bad_lines=False)

TypeError: read_csv() got an unexpected keyword argument 'error_bad_lines'

In [103]:
df_2011.head(3)

NameError: name 'df_2011' is not defined

In [ ]:
# Vamos juntar esses DataFrames
pd.concat([df_2011,df_2012,df_2013])

In [ ]:
# Podemos ver que existem 30832 linhas nesse DataFrame, o que corresponde a soma dos 3 DataFrames:

len(df_2011)+len(df_2012)+len(df_2013)

In [ ]:
# Mas temos um problema, como todos os dados estão concatenados, não sabemos qual é de que ano.
# Podemos utilizar o parâmetro 'keys' para sanar esse problema.

df_result=pd.concat([df_2011,df_2012,df_2013], keys=['2011','2012','2013'])
df_result

In [ ]:
df_result.index

In [ ]:
# Importante: A função concat() também possui os métodos 'inner' e 'outer'. Se quisermos juntar DataFrames que não possuem colunas idênticas.


# Pandas Idioms

Existem diferentes modos de solucionar um problema, mas uns são mais apropriados que outros. Os melhores jeitos de resolver 

são chamados de **Idiomatic Python**.

No caso do Pandas, chamamos esses jeitos de **pandorable**.

In [ ]:
import pandas as pd
import numpy as np
import timeit

df=pd.read_csv("datasets/census.csv")
df.head()


### chaining

In [ ]:
# O primeiro dos "Pandas idioms" é o método "chaining". A ideia geral é que podemos ir 'acorrentando' diferentes operações em um DataFrame.

# Aqui está um exemplo de um jeito 'pandorable' de escrever o código utilizando o método 'chaining'.
# Nessa código queremos excluir as linhas que apresentam 'SUMLEV' diferente de 50,  colocar os indices 'STNAME' e 'CTYNAME' 
# e renomear a coluna 'ESTIMATESBASE2010'.

In [ ]:
(df.where(df['SUMLEV']==50)
    
    .dropna()
    .set_index(['STNAME','CTYNAME'])
    .rename(columns={'ESTIMATESBASE2010': 'Estimates Base 2010'}))

In [ ]:
# Aqui um jeito não 'pandorable' de fazer a mesma coisa que a célula anterior.

# Primeiro criamos um dataframe a partir do original.
df =df[df['SUMLEV'] ==50 ]

#Depois setamos as novas colunas
df.set_index(['STNAME','CTYNAME'], inplace=True)

# E então renomeamos a coluna
df.rename(columns={'ESTIMATESBASE2010': 'Estimates Base 2010'})

##### timeit

Nós podemos passar essas abordagens para uma função e passar essa função na função 'timeit' para contar o tempo de execução, passando também o parâmetro 'number' que diz o número de vezes que a função deve ser executada.

In [ ]:
df = pd.read_csv('datasets/census.csv')

In [ ]:
# Primeira abordagem

def first_approach():
    return (df.where(df['SUMLEV']==50)
             .dropna()
             .set_index(['STNAME','CTYNAME'])
             .rename(columns={'ESTIMATESBASE2010': 'Estimates Base 2010'}))

timeit.timeit(first_approach, number=10)

In [ ]:
# Segunda abordagem

def second_approach():
    new_df = df[df['SUMLEV']==50]
    new_df.set_index(['STNAME','CTYNAME'], inplace=True)
    return new_df.rename(columns={'ESTIMATESBASE2010': 'Estimates Base 2010'})

timeit.timeit(second_approach, number=10)

In [ ]:
# Podemos ver que a segunda abordagem, a não 'pandorable', é mais rápida


### applymap

Python tem uma poderosa função chamada 'map', que aplica uma função em cada elemento de um iterável e retorna uma lista.

Pandas tem uma função semelhante chamada **applymap**. Nessa função, passamos uma função que deverá ser operada em cada célula do DataFrame e retorna um DataFrame.

No entanto, uma função mais usada é a função **apply** que aplica uma função ao longo de um eixo do DataFrame.

In [ ]:
# Vamos olhar para o DataFrame 'census'. Nesse DataFrame temos 5 colunas para a estimativa de uma população, cuja cada coluna corresponde a um ano.
import numpy as np
df=pd.read_csv('datasets/census.csv')
df.head(5)

In [ ]:
# É razoável que queiramos criar novas colunas com os valores máximos e mínimos ao longo dos anos para aquela região específica.
# Primeiro precisamos criar uma função que pega uma linha, ache o valor minimo e o valor máximo e então retorna uma nova linha de dados.

def min_max(row):  
    #data é a projeção da 'row' nas colunas selecionadas   
    data = row[['POPESTIMATE2010',
                'POPESTIMATE2011',
                'POPESTIMATE2012',
                'POPESTIMATE2013',
                'POPESTIMATE2014',
                'POPESTIMATE2015']]        
    return pd.Series({'min': np.min(data), 'max': np.max(data)})
df.apply(min_max, axis=1)
df

In [ ]:
# Se quisermos adicionar no DataFrame original, podemos fazer desse jeito:

def min_max(row):
    data = row[['POPESTIMATE2010',
                'POPESTIMATE2011',
                'POPESTIMATE2012',
                'POPESTIMATE2013',
                'POPESTIMATE2014',
                'POPESTIMATE2015']]
    # Create a new entry for max
    row['max'] = np.max(data)
    # Create a new entry for min
    row['min'] = np.min(data)
    return row
# Now just apply the function across the dataframe
df.apply(min_max, axis='columns')
df

In [ ]:
# Utilizando lambda:

rows = ['POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013','POPESTIMATE2014', 
        'POPESTIMATE2015']

df.apply(lambda x: np.max(x[rows]), axis=1).head() # x é cada linha do DataFrame, para calcular o np.max precisamos projetar as colunas de interesse já que as outras colunas possuem 'strings'.

In [ ]:
# Vamos criar uma função que divida os estados em regiões:  Northeast, Midwest, South, and West.

def get_state_region(x):
    northeast = ['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 
                 'Rhode Island','Vermont','New York','New Jersey','Pennsylvania']
    midwest = ['Illinois','Indiana','Michigan','Ohio','Wisconsin','Iowa',
               'Kansas','Minnesota','Missouri','Nebraska','North Dakota',
               'South Dakota']
    south = ['Delaware','Florida','Georgia','Maryland','North Carolina',
             'South Carolina','Virginia','District of Columbia','West Virginia',
             'Alabama','Kentucky','Mississippi','Tennessee','Arkansas',
             'Louisiana','Oklahoma','Texas']
    west = ['Arizona','Colorado','Idaho','Montana','Nevada','New Mexico','Utah',
            'Wyoming','Alaska','California','Hawaii','Oregon','Washington']
    
    if x in northeast:
        return "Northeast"
    elif x in midwest:
        return "Midwest"
    elif x in south:
        return "South"
    else:
        return "West"
    


In [ ]:
# Agora que temos a função, vamos dizer que queremos criar uma nova coluna chamada 'Region'.
df['state_region']=df['STNAME'].apply(lambda x: get_state_region(x))

In [ ]:
# Agora vamos ver os resultados
df[['STNAME','state_region']]

# Groupby
A ideia por trás da função groupby() é que ela pega um dataframe e separa ele em pedaços baseados em alguma chave/valor.

### Spliting

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Vamos olhar o DataFrame census
df=pd.read_csv('datasets/census.csv')
df=df[df['SUMLEV']==50]
df.head()

In [ ]:
%%timeit -n 3

for state in df['STNAME'].unique():
    avg=np.average(df[df['STNAME']==state]['CENSUS2010POP'])
    print('Counties in state ' + state + 
          ' have an average population of ' + str(avg))

In [ ]:
# Agora vamos tentar pelo método groupby()

#Precisamos dizer para o Pandas que estamos interessados em agrupar pelo nome do estado, isso é o 'split'
for result in df.groupby('STNAME'):
    firstState=result
    break
print(type(firstState))
print(firstState[0])
firstState[1]

In [ ]:
# Podemos ver que a função groupby cria um objeto com tuplas, e em cada tupla temos o nome do estado na primeira posição e o 
# DataFrame contendo as informações do estado na segunda posição.


In [ ]:
import numpy as np

In [ ]:
# %%timeit
# # Voltando ao cálculo da média
# for group,frame in df.groupby("STNAME"):
#     avg=np.average(frame['CENSUS2010POP'])
#     print('Counties in state ' + group + 
#           ' have an average population of ' + str(avg))

Counties in state Alabama have an average population of 71339.34328358209
Counties in state Alaska have an average population of 24490.724137931036
Counties in state Arizona have an average population of 426134.4666666667
Counties in state Arkansas have an average population of 38878.90666666667
Counties in state California have an average population of 642309.5862068966
Counties in state Colorado have an average population of 78581.1875
Counties in state Connecticut have an average population of 446762.125
Counties in state Delaware have an average population of 299311.3333333333
Counties in state District of Columbia have an average population of 601723.0
Counties in state Florida have an average population of 280616.5671641791
Counties in state Georgia have an average population of 60928.63522012578
Counties in state Hawaii have an average population of 272060.2
Counties in state Idaho have an average population of 35626.86363636364
Counties in state Illinois have an average populat

Counties in state Alabama have an average population of 71339.34328358209
Counties in state Alaska have an average population of 24490.724137931036
Counties in state Arizona have an average population of 426134.4666666667
Counties in state Arkansas have an average population of 38878.90666666667
Counties in state California have an average population of 642309.5862068966
Counties in state Colorado have an average population of 78581.1875
Counties in state Connecticut have an average population of 446762.125
Counties in state Delaware have an average population of 299311.3333333333
Counties in state District of Columbia have an average population of 601723.0
Counties in state Florida have an average population of 280616.5671641791
Counties in state Georgia have an average population of 60928.63522012578
Counties in state Hawaii have an average population of 272060.2
Counties in state Idaho have an average population of 35626.86363636364
Counties in state Illinois have an average populat

Counties in state Alabama have an average population of 71339.34328358209
Counties in state Alaska have an average population of 24490.724137931036
Counties in state Arizona have an average population of 426134.4666666667
Counties in state Arkansas have an average population of 38878.90666666667
Counties in state California have an average population of 642309.5862068966
Counties in state Colorado have an average population of 78581.1875
Counties in state Connecticut have an average population of 446762.125
Counties in state Delaware have an average population of 299311.3333333333
Counties in state District of Columbia have an average population of 601723.0
Counties in state Florida have an average population of 280616.5671641791
Counties in state Georgia have an average population of 60928.63522012578
Counties in state Hawaii have an average population of 272060.2
Counties in state Idaho have an average population of 35626.86363636364
Counties in state Illinois have an average populat

Counties in state Alabama have an average population of 71339.34328358209
Counties in state Alaska have an average population of 24490.724137931036
Counties in state Arizona have an average population of 426134.4666666667
Counties in state Arkansas have an average population of 38878.90666666667
Counties in state California have an average population of 642309.5862068966
Counties in state Colorado have an average population of 78581.1875
Counties in state Connecticut have an average population of 446762.125
Counties in state Delaware have an average population of 299311.3333333333
Counties in state District of Columbia have an average population of 601723.0
Counties in state Florida have an average population of 280616.5671641791
Counties in state Georgia have an average population of 60928.63522012578
Counties in state Hawaii have an average population of 272060.2
Counties in state Idaho have an average population of 35626.86363636364
Counties in state Illinois have an average populat

Counties in state Alabama have an average population of 71339.34328358209
Counties in state Alaska have an average population of 24490.724137931036
Counties in state Arizona have an average population of 426134.4666666667
Counties in state Arkansas have an average population of 38878.90666666667
Counties in state California have an average population of 642309.5862068966
Counties in state Colorado have an average population of 78581.1875
Counties in state Connecticut have an average population of 446762.125
Counties in state Delaware have an average population of 299311.3333333333
Counties in state District of Columbia have an average population of 601723.0
Counties in state Florida have an average population of 280616.5671641791
Counties in state Georgia have an average population of 60928.63522012578
Counties in state Hawaii have an average population of 272060.2
Counties in state Idaho have an average population of 35626.86363636364
Counties in state Illinois have an average populat

...

Counties in state Alabama have an average population of 71339.34328358209
Counties in state Alaska have an average population of 24490.724137931036
Counties in state Arizona have an average population of 426134.4666666667
Counties in state Arkansas have an average population of 38878.90666666667
Counties in state California have an average population of 642309.5862068966
Counties in state Colorado have an average population of 78581.1875
Counties in state Connecticut have an average population of 446762.125
Counties in state Delaware have an average population of 299311.3333333333
Counties in state District of Columbia have an average population of 601723.0
Counties in state Florida have an average population of 280616.5671641791
Counties in state Georgia have an average population of 60928.63522012578
Counties in state Hawaii have an average population of 272060.2
Counties in state Idaho have an average population of 35626.86363636364
Counties in state Illinois have an average populat

Counties in state Alabama have an average population of 71339.34328358209
Counties in state Alaska have an average population of 24490.724137931036
Counties in state Arizona have an average population of 426134.4666666667
Counties in state Arkansas have an average population of 38878.90666666667
Counties in state California have an average population of 642309.5862068966
Counties in state Colorado have an average population of 78581.1875
Counties in state Connecticut have an average population of 446762.125
Counties in state Delaware have an average population of 299311.3333333333
Counties in state District of Columbia have an average population of 601723.0
Counties in state Florida have an average population of 280616.5671641791
Counties in state Georgia have an average population of 60928.63522012578
Counties in state Hawaii have an average population of 272060.2
Counties in state Idaho have an average population of 35626.86363636364
Counties in state Illinois have an average populat

Counties in state Alabama have an average population of 71339.34328358209
Counties in state Alaska have an average population of 24490.724137931036
Counties in state Arizona have an average population of 426134.4666666667
Counties in state Arkansas have an average population of 38878.90666666667
Counties in state California have an average population of 642309.5862068966
Counties in state Colorado have an average population of 78581.1875
Counties in state Connecticut have an average population of 446762.125
Counties in state Delaware have an average population of 299311.3333333333
Counties in state District of Columbia have an average population of 601723.0
Counties in state Florida have an average population of 280616.5671641791
Counties in state Georgia have an average population of 60928.63522012578
Counties in state Hawaii have an average population of 272060.2
Counties in state Idaho have an average population of 35626.86363636364
Counties in state Illinois have an average populat

Counties in state Alabama have an average population of 71339.34328358209
Counties in state Alaska have an average population of 24490.724137931036
Counties in state Arizona have an average population of 426134.4666666667
Counties in state Arkansas have an average population of 38878.90666666667
Counties in state California have an average population of 642309.5862068966
Counties in state Colorado have an average population of 78581.1875
Counties in state Connecticut have an average population of 446762.125
Counties in state Delaware have an average population of 299311.3333333333
Counties in state District of Columbia have an average population of 601723.0
Counties in state Florida have an average population of 280616.5671641791
Counties in state Georgia have an average population of 60928.63522012578
Counties in state Hawaii have an average population of 272060.2
Counties in state Idaho have an average population of 35626.86363636364
Counties in state Illinois have an average populat

Counties in state Alabama have an average population of 71339.34328358209
Counties in state Alaska have an average population of 24490.724137931036
Counties in state Arizona have an average population of 426134.4666666667
Counties in state Arkansas have an average population of 38878.90666666667
Counties in state California have an average population of 642309.5862068966
Counties in state Colorado have an average population of 78581.1875
Counties in state Connecticut have an average population of 446762.125
Counties in state Delaware have an average population of 299311.3333333333
Counties in state District of Columbia have an average population of 601723.0
Counties in state Florida have an average population of 280616.5671641791
Counties in state Georgia have an average population of 60928.63522012578
Counties in state Hawaii have an average population of 272060.2
Counties in state Idaho have an average population of 35626.86363636364
Counties in state Illinois have an average populat

### Using functions to split our database

In [ ]:
# Podemos passar uma função como parâmetro da função groupby() para segmentar os nossos dados.

# Suponhamos que queremos trabalhar em apenas 1/3 o DataFrame. Podemos criar uma função que retorna um número entre 0 e 2
# baseado no primeiro caractere do nome do estado.
# Então podemos dizer ao groupby para usar essa função para segmentar os nossos dados.
# Importante: Precisamos setar o indice como a coluna que iremos agrupar
df=df.reset_index()
df=df.set_index('STNAME')

# Essa função retorna uma variável que será o nome do grupo, nesse caso 0,1,2.
# O groupby irá armazenar as linhas do DataFrame nesses grupos de acordo com a função.
def set_batch_number(item):
    if item[0]<'M':
        return 0
    if item[0]<'Q':
        return 1
    return 2

for group,frame in df.groupby(set_batch_number):
    print('There are ' + str(len(frame)) + ' records in group ' + str(group) + ' for processing.')
    print(group)
    print(frame)

In [ ]:
# Nesse exemplo, temos os estados com o primeiro caractere de A até L incluídos no grupo 1, de M até P incluídos no grupo 2 e 
# o resto no grupo 3.

# Se nenhuma coluna for passada, groupby irá usar a coluna index automaticamente.

### Another example

In [ ]:
# Nesse exemplo, queremos usar o dataset de casas do airbnb.
# Aqui temos duas colunas de interesse, a 'cancellation_policy' e a 'review_scores_value'.
import pandas as pd
df=pd.read_csv('datasets/listings.csv')
df.head()

In [ ]:
# Então, como podemos agrupar com essas duas colunas?
# Podemos fazer de dois jeitos

# A primeira abordagem seria promover ambas como multi-index e chamar o groupby() utilizando o parâmetro 'level' passando (0,1).
# A segunda abordagem seria passar como parâmetro do groupby as colunas dentro de uma lista: df.groupby(["cancellation_policy","review_scores_value"]):


In [ ]:
# Vamos fazer do primeiro jeito  
df=df.reset_index()
df=df.set_index(["cancellation_policy","review_scores_value"])
for result in df.groupby(level=(0,1)):
    print(result[0])
    break
    

In [ ]:
df

In [ ]:
# No caso multi-index, o primeiro elemento da tupla é uma tupla contendo os nomes dos indices

In [ ]:
# Voltando ao exemplo:
for group, frame in df.groupby(level=(0,1)):
    print(group)

In [ ]:
# E se quisermos agrupar nas colunas 'cancelation policy' e 'review scores', mas separar a coluna 'review scores' em 2 casos: igual e diferente de 10.
# Nesse caso fazemos o uso de funções novamente

def grouping_fun(item):
    # Check the "review_scores_value" portion of the index. item is in the format of
    # (cancellation_policy,review_scores_value)
    print(f'k {item}')
    if item[1] == 10.0:
        return (item[0],"10.0")
    else:
        return (item[0],"not 10.0")
    
for group, frame in df.groupby(grouping_fun):
    print(group)

# Applying
Existem 3 categorias de processamento de dados: 'aggregation', 'transformation' e 'filtration'.


### Aggregation

In [ ]:
df['cancellation_policy']

In [ ]:
# Utilizamos o método agg() no objeto groupby().
# Podemos passar como parâmetro do agg() um dicionário com as colunas que queremos e a função que desejamos aplicar a elas.
df=pd.read_csv('datasets/listings.csv')
df=df.reset_index()

#Vamos agrupar por 'cancellation_policy' e achar a média da coluna 'review_scores'.
df.groupby("cancellation_policy").agg({"review_scores_value":np.average})

In [ ]:
# Isso aconteceu porque 'np.average' não ignora os NaN, precisamos usar a função 'np.nanmean' ou 'np.mean'
df.groupby("cancellation_policy").agg({"review_scores_value":np.nanmean})

In [ ]:
# Podemos extender o dicionário passando múltiplas funções ou colunas

df.groupby("cancellation_policy").agg({"review_scores_value":(np.nanmean,np.nanstd),
                                      "reviews_per_month":np.nanmean})

### Transformation

In [ ]:
# Enquanto agg() retorna um único valor por coluna,ou uma única linha por DataFrame/grupo.
# transform() retorna um objeto do mesmo tamanho do grupo, isso torna a combinação de dados mais fácil.

df=pd.read_csv('datasets/listings.csv')
# Exemplo: Vamos supor que queiramos incluir a média dos valores, mas preservar o DataFrame
cols=['cancellation_policy','review_scores_value']
df[cols].groupby('cancellation_policy').transform(np.mean)

In [ ]:
# Aqui, os indices são iguais ao DataFrame original, então podemos junta-los.
transform_df=df[cols].groupby('cancellation_policy').transform(np.mean)
transform_df.rename({'review_scores_value':'mean_review_scores'}, axis='columns', inplace=True)

df=df.merge(transform_df, left_index=True, right_index=True)
df[['cancellation_policy','mean_review_scores']]
df

### Filtering
A função filter() recebe uma função que é aplicada ao DataFrame e retorna True ou False

In [ ]:
df

In [ ]:
# Por exemplo, se quisermos a média acima de 9 nos nossos resultados
filtering_df=df.groupby('cancellation_policy').filter(lambda x: np.nanmean(x['review_scores_value'])>9.2)
filtering_df['cancellation_policy'].unique()


### Applying
A função apply() permite aplicar uma função arbitrária em cada grupo, e retornar o resultado em um único DataFrame onde o indice é preservado.

In [ ]:
import pandas as pd
df=pd.read_csv("datasets/listings.csv")
df=df[['cancellation_policy','review_scores_value']]
df.head()

In [ ]:
# Vamos calcular a média e o desvio padrão

def calc_mean_review_scores(group):
    #group é o DataFrame   
    avg=np.nanmean(group["review_scores_value"])   
    group['mean_diff']=np.abs(avg-group["review_scores_value"])
    return group
df.groupby('cancellation_policy').apply(calc_mean_review_scores).head()

# Scales
Pandas suporta diferentes tipos de dados como string,inteiros,floats.
O que não é capturado é o que chamamos de **escala dos dados**.

Exemplo: Notas A+, A, A-, B+, B, B- de um estudante não tem a mesma escala. A diferença entre as notas A e A- e a diferença entre as notas A- e B+ não são iguais.

Existem pelo menos 4 tipos diferentes de escala:


**Ratio scale**

-As unidades de medida são igualmente espaçadas e operações matemáticas são válidas.

Exemplo: Peso e Altura

**Interval Scale**

-As unidades de medida são igualmente espaçadas, mas não existe uma clara ausência de valor e as operações matemáticas não são válidas.

Exemplo: Temperatura -> Não existe uma ausência de temperatura, 0° representa uma temperatura. Outro exemplo é a bússola, não existe ausência de direção.


**Ordinal Scale**

-A ordem dos valores é importante, mas a diferença entre os valores não é igualmente espaçada

Exemplo: Notas de um aluno (A+,A,A-)

**Nominal Scale**

-Também chamada de **categorical data**.

-Exemplo : Times de futebol; tem um número limitado de times de futebol, mas mudar a ordem deles ou aplicar operações matemáticas não tem sentido

In [ ]:
import pandas as pd

df=pd.DataFrame(['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D'],
                index=['excellent', 'excellent', 'excellent', 'good', 'good', 'good', 
                       'ok', 'ok', 'ok', 'poor', 'poor'],
               columns=["Grades"])
df

In [ ]:
# Podemos dizer ao Pandas que queremos mudar o 'type' para 'category' usando a função astype()
df["Grades"].astype("category").head()

In [ ]:
# Podemos ver que o 'dtype' mudou para 'category' e temos 11 categorias.

### Boolean Mask

In [ ]:
# O que acontece se quisermos pegar as notas desse aluno que foram maiores do que C?
df[df["Grades"]>"C"]

In [ ]:
# Errado, o Pandas não reconhece a ordem dos dados

In [ ]:
# Podemos dizer ao Pandas também, que nossos dados são ordenados criando um parâmetro com a ordem e 'ordered=True'.

my_categories=pd.CategoricalDtype(categories=['D', 'D+', 'C-', 'C', 'C+', 'B-', 'B', 'B+', 'A-', 'A', 'A+'],ordered=True)

# E então, podemos passar como parâmetro para a função astype().
grades=df['Grades'].astype(my_categories)
grades.head

In [ ]:
df[df["Grades"]>"C"]

In [ ]:
# Agora, Pandas não está se preocupando somente com as 11 categorias, mas também com a ordem delas.

In [ ]:
# Então, agora sim podemos fazer a consulta das notas maiores do que 'C'.
grades[grades>"C"]

### Another scale-based operation

Converte a escala de algo que está na escala de intervalo ou razão,como uma nota numérica, em uma escala que é 'categorical'.
Exemplo: Se estivermos visualizando frequências de categorias.

Pandas tem uma função chamada cut, que recebe como argumento alguma estrutura tipo array como uma coluna de um dataframe ou um
Serie

In [ ]:
# Vamos pegar o dataset Census.
# Vimos que poderíamos agrupar por estado e depois agregar para obter um lista do tamanho médio do condado por estado.

import numpy as np
df=pd.read_csv("datasets/census.csv")

df=df[df['SUMLEV']==50]

df=df.groupby('STNAME')['CENSUS2010POP'].agg(np.average)
df.head()

In [ ]:
# Agora, podemos usar o cut, colocando bins=10
pd.cut(df,10)

In [ ]:
# Aqui vemos que estados como o alabama e o alasca se enquadram na mesma categoria, enquanto a Califórnia e o distrito de columbia
# se enquadram em uma categoria muito diferente. 
# Agora, cortar é apenas uma maneira de criar categorias a partir de seus dados, e existem muitos outros métodos. 
# Por exemplo, cut fornece dados de intervalo, onde o espaçamento entre cada categoria é igual. 
# Mas às vezes você quer formar categorias com base na frequência – você quer que o número de itens em cada caixa seja o mesmo,
# em vez do espaçamento entre as caixas. Isso realmente depende da forma dos seus dados e do que você planeja fazer com eles.

# Pivot Table
Um **pivot table** é um jeito de resumir os dados para um propósito particular. Se faz necessário o uso da função agg().
Um **pivot table** é um DataFrame, onde as linhas representam uma variável que estamos interessados,as colunas outra variável e as células são valores agregados.

In [ ]:
import pandas as pd
import numpy as np

In [126]:
# Vamos trabalhar com o dataset "Times Higher Education World University Ranking", que mostra o ranking das universidades
df = pd.read_csv('datasets/cwurData.csv')
df.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012


In [127]:
# Vamos supor que queremos criar uma nova coluna chamada "Rank_level" ondas instituições com "world_rank" entre 1 e 100 são 
# categorizadas como 'first tier', as com "world_rank" entre 101 e 200 serão categorizadas como 'second tier' e assim por diante.

# Uma das soluções que vimos é:
def create_category(ranking):    
    if (ranking >= 1) & (ranking <= 100):
        return "First Tier Top University"
    elif (ranking >= 101) & (ranking <= 200):
        return "Second Tier Top University"
    elif (ranking >= 201) & (ranking <= 300):
        return "Third Tier Top University"
    return "Other Top Unversity"

df['Rank_Level'] = df['world_rank'].apply(lambda x: create_category(x))

df[['Rank_Level','score','country']]

,Rank_Level,score,country
0,First Tier Top University,100.00,USA
1,First Tier Top University,91.67,USA
2,First Tier Top University,89.50,USA
3,First Tier Top University,86.17,United Kingdom
4,First Tier Top University,85.21,USA
...,...,...,...
2195,Other Top Unversity,44.03,Portugal
2196,Other Top Unversity,44.03,Egypt
2197,Other Top Unversity,44.03,Brazil
2198,Other Top Unversity,44.02,Spain


In [129]:
df

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year,Rank_Level
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012,First Tier Top University
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012,First Tier Top University
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012,First Tier Top University
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012,First Tier Top University
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012,First Tier Top University
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2195,996,University of the Algarve,Portugal,7,367,567,218,926,845,812,969.0,816,44.03,2015,Other Top Unversity
2196,997,Alexandria University,Egypt,4,236,566,218,997,908,645,981.0,871,44.03,2015,Other Top Unversity
2197,998,Federal University of Ceará,Brazil,18,367,549,218,830,823,812,975.0,824,44.03,2015,Other Top Unversity
2198,999,University of A Coruña,Spain,40,367,567,218,886,974,812,975.0,651,44.02,2015,Other Top Unversity


In [128]:
# Um pivot table nos permite dinamizar uma dessas colunas em um novo cabeçalho de coluna e compará-la com 
# outra coluna como índices de linha

#Exemplo
# Vamos comparar o 'rank_level' com os países em termos do 'overall score'
#Ou seja, nesse caso queremos que nosso indice sejam os países, o cabeçalho seja o rank level e os valores das células a média do 'overall score'.

df.pivot_table(values='score', index='country', columns='Rank_Level', aggfunc=[np.mean]).head()

C:\Users\joao\AppData\Local\Temp\ipykernel_16132\3593368816.py:8: FutureWarning: The provided callable <function mean at 0x0000020D8B4F6FC0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df.pivot_table(values='score', index='country', columns='Rank_Level', aggfunc=[np.mean]).head()


mean                      \
Rank_Level First Tier Top University Other Top Unversity   
country                                                    
Argentina                        NaN           44.672857   
Australia                    47.9425           44.645750   
Austria                          NaN           44.864286   
Belgium                      51.8750           45.081000   
Brazil                           NaN           44.499706   

                                                                 
Rank_Level Second Tier Top University Third Tier Top University  
country                                                          
Argentina                         NaN                       NaN  
Australia                     49.2425                 47.285000  
Austria                           NaN                 47.066667  
Belgium                       49.0840                 46.746667  
Brazil                        49.5650                       NaN

In [ ]:
#IMPORTANTE: o parâmetro 'aggfunc' é igual a 'mean' por default,então nesse caso não precisaríamos passar o 'np.mean'

In [ ]:
# Podemos ver um DataFrame hierárquico onde os indices(ou linhas) são por países, e as linhas possuem 2 levels: o level mais acima
# indicando 'mean' e o segundo os 'ranks'

In [ ]:
# Podemos aplicar mais de uma função no parâmetro 'aggfunc'.
# Vamos tentar

df.pivot_table(values='score', index='country', columns='Rank_Level', aggfunc=[np.mean, np.max]).head()

In [ ]:
# Agora temos a média e o valor máximo. 
# Se quisermos ver uma média das médias, e o máximo dos valores máximos, podemos indicar utilizando o parâmetro margins=True.
df.pivot_table(values='score', index='country', columns='Rank_Level', aggfunc=[np.mean, np.max], 
               margins=True).head()

In [ ]:
# Pivot table é só um DataFrame multi-level, então podemos acessar as células de uma maneira semelhante a que faziamos antes.
new_df=df.pivot_table(values='score', index='country', columns='Rank_Level', aggfunc=[np.mean, np.max], 
               margins=True)
new_df.head()

In [ ]:
new_df.unstack().unstack()

In [104]:
# Vamos olhar os indices
print(new_df.index)

# Vamos olhar as colunas
print(new_df.columns)

Index(['All', 'Argentina', 'Australia', 'Austria', 'Belgium', 'Brazil',
       'Bulgaria', 'Canada', 'Chile', 'China', 'Colombia', 'Croatia', 'Cyprus',
       'Czech Republic', 'Denmark', 'Egypt', 'Estonia', 'Finland', 'France',
       'Germany', 'Greece', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Iran',
       'Ireland', 'Israel', 'Italy', 'Japan', 'Lebanon', 'Lithuania',
       'Malaysia', 'Mexico', 'Netherlands', 'New Zealand', 'Norway', 'Poland',
       'Portugal', 'Puerto Rico', 'Romania', 'Russia', 'Saudi Arabia',
       'Serbia', 'Singapore', 'Slovak Republic', 'Slovenia', 'South Africa',
       'South Korea', 'Spain', 'Sweden', 'Switzerland', 'Taiwan', 'Thailand',
       'Turkey', 'USA', 'Uganda', 'United Arab Emirates', 'United Kingdom',
       'Uruguay'],
      dtype='object', name='country')
MultiIndex([('mean',  'First Tier Top University'),
            ('mean',        'Other Top Unversity'),
            ('mean', 'Second Tier Top University'),
            ('mean',  'Thir

In [105]:
# Podemos ver que as colunas são hierárquicas.
# A coluna mais alta possui duas categorias: mean e max; a coluna mais baixa possui 4 categorias que são os 4 'rank levels'.

# Como fazer para acessar as células?
# Exemplo: Como acessar a coluna da média do rank 'First Tier Top University'
# Precisamos fazer duas projeções, a primeira na 'mean' e a segunda na coluna 'First Tier Top University'
new_df['mean']['First Tier Top University'].head()

country
All          58.350675
Argentina          NaN
Australia    47.942500
Austria            NaN
Belgium      51.875000
Name: First Tier Top University, dtype: float64

In [106]:
# E se quisermos achar o país (indice) que possui a maior média de 'overallscore' na coluna 'First Tier Top University'?
# Podemos usar a função idxmax()

np.max(new_df['mean']['First Tier Top University']) # nos retorna a média

new_df['mean']['First Tier Top University'].idxmax() # nos retorna o indica cuja a média é máxima

'United Kingdom'

### Stack and Unstack

In [107]:
# Se quisermos mudar o 'shape' do 'pivot table', podemos usar as funções stack e unstack.
# "Stacking" gira o cabeçalho mais baixa para se tornar o índice mais interno.
# "Unstacking" gira o índice mais interno para se tornar o cabeçalho mais baixo

In [108]:
new_df.head()

mean                      \
Rank_Level First Tier Top University Other Top Unversity   
country                                                    
All                        58.350675           44.738871   
Argentina                        NaN           44.672857   
Australia                  47.942500           44.645750   
Austria                          NaN           44.864286   
Belgium                    51.875000           45.081000   

                                                                            \
Rank_Level Second Tier Top University Third Tier Top University        All   
country                                                                      
All                          49.06545                 46.843450  47.798395   
Argentina                         NaN                       NaN  44.672857   
Australia                    49.24250                 47.285000  45.825517   
Austria                           NaN                 47.066667  45.139583   
Belgium                      49.08400                 46.746667  47.011000   

                                 max                      \
Rank_Level First Tier Top University Other Top Unversity   
country                                                    
All                           100.00               46.34   
Argentina                        NaN               45.66   
Australia                      51.61               45.97   
Austria                          NaN               46.29   
Belgium                        52.03               46.21   

                                                                         
Rank_Level Second Tier Top University Third Tier Top University     All  
country                                                                  
All                             51.29                     47.93  100.00  
Argentina                         NaN                       NaN   45.66  
Australia                       50.40                     47.47   51.61  
Austria                           NaN                     47.78   47.78  
Belgium                         49.73                     47.14   52.03

In [109]:
new_df.unstack().unstack()

country                                 All  Argentina  Australia    Austria  \
     Rank_Level                                                                
mean First Tier Top University    58.350675        NaN  47.942500        NaN   
     Other Top Unversity          44.738871  44.672857  44.645750  44.864286   
     Second Tier Top University   49.065450        NaN  49.242500        NaN   
     Third Tier Top University    46.843450        NaN  47.285000  47.066667   
     All                          47.798395  44.672857  45.825517  45.139583   
max  First Tier Top University   100.000000        NaN  51.610000        NaN   
     Other Top Unversity          46.340000  45.660000  45.970000  46.290000   
     Second Tier Top University   51.290000        NaN  50.400000        NaN   
     Third Tier Top University    47.930000        NaN  47.470000  47.780000   
     All                         100.000000  45.660000  51.610000  47.780000   

country                            Belgium     Brazil  Bulgaria     Canada  \
     Rank_Level                                                              
mean First Tier Top University   51.875000        NaN       NaN  53.633846   
     Other Top Unversity         45.081000  44.499706    44.335  44.760541   
     Second Tier Top University  49.084000  49.565000       NaN  49.218182   
     Third Tier Top University   46.746667        NaN       NaN  46.826364   
     All                         47.011000  44.781111    44.335  47.359306   
max  First Tier Top University   52.030000        NaN       NaN  60.870000   
     Other Top Unversity         46.210000  46.080000    44.480  45.740000   
     Second Tier Top University  49.730000  49.820000       NaN  51.230000   
     Third Tier Top University   47.140000        NaN       NaN  47.690000   
     All                         52.030000  49.820000    44.480  60.870000   

country                            Chile      China  ...     Sweden  \
     Rank_Level                                      ...              
mean First Tier Top University       NaN  53.592500  ...  50.672000   
     Other Top Unversity         44.7675  44.564267  ...  45.272500   
     Second Tier Top University      NaN  47.868000  ...  49.306250   
     Third Tier Top University       NaN  46.926250  ...  46.456667   
     All                         44.7675  44.992575  ...  47.890000   
max  First Tier Top University       NaN  55.300000  ...  53.640000   
     Other Top Unversity         45.3300  45.920000  ...  46.320000   
     Second Tier Top University      NaN  48.140000  ...  50.340000   
     Third Tier Top University       NaN  47.760000  ...  46.790000   
     All                         45.3300  55.300000  ...  53.640000   

country                          Switzerland     Taiwan   Thailand  Turkey  \
     Rank_Level                                                              
mean First Tier Top University     54.005000  54.210000        NaN     NaN   
     Other Top Unversity           44.625000  44.476667  44.830000  44.481   
     Second Tier Top University    48.184000        NaN        NaN     NaN   
     Third Tier Top University     47.930000  47.065000  46.550000     NaN   
     All                           51.208846  45.012391  45.116667  44.481   
max  First Tier Top University     72.180000  54.230000        NaN     NaN   
     Other Top Unversity           45.050000  45.490000  45.860000  45.570   
     Second Tier Top University    49.170000        NaN        NaN     NaN   
     Third Tier Top University     47.930000  47.160000  46.550000     NaN   
     All                           72.180000  54.230000  46.550000  45.570   

country                                 USA  Uganda  United Arab Emirates  \
     Rank_Level                                                             
mean First Tier Top University    61.066726     NaN                   NaN   
     Other Top Unversity          44.871718   44.28                 44.22   
    

In [110]:
# Utilizando a função 'stack', iremos mover o cabeçalho mais baixo (rank) para ser o indice mais interno.
new_df=new_df.stack()
new_df.head()

mean     max
country Rank_Level                                   
All     First Tier Top University   58.350675  100.00
        Other Top Unversity         44.738871   46.34
        Second Tier Top University  49.065450   51.29
        Third Tier Top University   46.843450   47.93
        All                         47.798395  100.00

In [111]:
# Utilizando a função 'unstack', iremos mover o indice mais interno(rank) para o cabeçalho mais interno
new_df=new_df.unstack()
new_df.head()

mean                      \
Rank_Level First Tier Top University Other Top Unversity   
country                                                    
All                        58.350675           44.738871   
Argentina                        NaN           44.672857   
Australia                  47.942500           44.645750   
Austria                          NaN           44.864286   
Belgium                    51.875000           45.081000   

                                                                            \
Rank_Level Second Tier Top University Third Tier Top University        All   
country                                                                      
All                          49.06545                 46.843450  47.798395   
Argentina                         NaN                       NaN  44.672857   
Australia                    49.24250                 47.285000  45.825517   
Austria                           NaN                 47.066667  45.139583   
Belgium                      49.08400                 46.746667  47.011000   

                                 max                      \
Rank_Level First Tier Top University Other Top Unversity   
country                                                    
All                           100.00               46.34   
Argentina                        NaN               45.66   
Australia                      51.61               45.97   
Austria                          NaN               46.29   
Belgium                        52.03               46.21   

                                                                         
Rank_Level Second Tier Top University Third Tier Top University     All  
country                                                                  
All                             51.29                     47.93  100.00  
Argentina                         NaN                       NaN   45.66  
Australia                       50.40                     47.47   51.61  
Austria                           NaN                     47.78   47.78  
Belgium                         49.73                     47.14   52.03

# Date/Time Functionality

-Timestamp

-Period

-DatetimeIndex

-PeriodIndex

### Timestamp

In [112]:
import pandas as pd
# Representa uma marca temporal, associada com um único ponto no tempo.

# Por exemplo, vamos criar um timestamp usando a string 9/1/2019 10:05AM.
pd.Timestamp('9/1/2019 10:05AM')

Timestamp('2019-09-01 10:05:00')

In [113]:
# Podemos criar um timestamp passando multíplos parâmetros, como ano,mês,data,hora e minutos, separadamente.
pd.Timestamp(2019,12,20,0,0)
pd.Timestamp(20)

Timestamp('1970-01-01 00:00:00.000000020')

In [114]:
# O atributo 'isoweekday()' retorna o dia da semana de um timestamp. 1 representa "Monday" e 7 "Sunday".
pd.Timestamp(2019, 12, 20, 0, 0).isoweekday()

5

In [115]:
# Podemos extrair o ano, mês, dia, hora, minuto e segundo de um timestamp
print(pd.Timestamp(2019, 12, 20, 5, 2,23).year)
print(pd.Timestamp(2019, 12, 20, 5, 2,23).month)
print(pd.Timestamp(2019, 12, 20, 5, 2,23).day)
print(pd.Timestamp(2019, 12, 20, 5, 2,23).hour)
print(pd.Timestamp(2019, 12, 20, 5, 2,23).minute)
print(pd.Timestamp(2019, 12, 20, 5, 2,23).second)


2019
12
20
5
2
23


### Period

In [116]:
# Supondo que não estamos interessados em um ponto do tempo, mas em um intervalo de tempo. 
# Periodo representa um único intervalo de tempo, como um dia ou um mês

# Podemos criar o período representando o mês Janeiro do ano de 2016.
pd.Period('1/2016')

Period('2016-01', 'M')

In [117]:
# Podemos ver que a 'classe' do período é retornada também, nesse caso 'M'(month)

#Outro exemplo:
pd.Period('3/5/2016')

Period('2016-03-05', 'D')

In [118]:
# Podemos aplicar operações aritméticas (baseadas na 'classe'): como temos "M", então queremos dizer +15 meses
pd.Period('1/2016') + 15

Period('2017-04', 'M')

In [119]:
# Outro exemplo
pd.Period('3/5/2016') - 2

Period('2016-03-03', 'D')

### DatetimeIndex and PeriodIndex

In [120]:
# O índice de um timestamp é chamado 'DatetimeIndex'.
# Vamos ver um exemplo:

# Vamos criar uma série t1, com indices iguais a timestamp
t1 = pd.Series(list('abc'), [pd.Timestamp('2016-09-01'), pd.Timestamp('2016-09-02'), 
                             pd.Timestamp('2016-09-03')])
t1

2016-09-01    a
2016-09-02    b
2016-09-03    c
dtype: object

In [121]:
type(t1.index) # DatetimeIndex

pandas.core.indexes.datetimes.DatetimeIndex

In [122]:
# Similarmente para o período
t2 = pd.Series(list('def'), [pd.Period('2016-09'), pd.Period('2016-10'), 
                             pd.Period('2016-11')])
t2

2016-09    d
2016-10    e
2016-11    f
Freq: M, dtype: object

In [123]:
type(t2.index)

pandas.core.indexes.period.PeriodIndex

### Converting to Datetime

In [124]:
import numpy as np
# Vamos supor que tenhamos uma lista de datas no formato string e queremos criar um DataFrame

#Vamos utilizar diferentes formatos como exemplo.
d1 = ['2 June 2013', 'Aug 29, 2014', '2015-06-26', '7/12/16']

ts3 = pd.DataFrame(np.random.randint(10, 100, (4,2)), index=d1, 
                   columns=list('ab'))
ts3

,a,b
2 June 2013,58,78
"Aug 29, 2014",25,34
2015-06-26,78,64
7/12/16,53,23


In [125]:
# Utilizando 'to_datetime', pandas irá tentar converter os valores para Datetime.

ts3.index = pd.to_datetime(ts3.index)
ts3

ValueError: time data "Aug 29, 2014" doesn't match format "%d %B %Y", at position 1. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
# 'to_datetime' tbm tem atributos, podemos passar 'dayfirst=True' para formatar no tipo europeu.
pd.to_datetime('4.7.12', dayfirst=True)

### Timedelta


In [ ]:
# Timedeltas são diferenças de tempo.
# Se quisermos a diferença entre 3 de Setembro e 1 de Setembro, pegaremos um Timedelta de 2 dias.
pd.Timestamp('9/3/2016')-pd.Timestamp('9/1/2016')

In [ ]:
# Podemos também querer saber qual data e tempo será daqui 12 dias e 3 horas a partir do dia 2 de Setembro.
pd.Timestamp('9/2/2016 8:10AM') + pd.Timedelta('12D 3H')

### Offset

In [ ]:
# Offset é similar ao timedelta, mas segue regras específicas do calendário.
# Exemplo: dia útil, fim do mês.

# Vamos criar um timestamp e ver qual dia da semana é
pd.Timestamp('9/4/2016').weekday()

In [ ]:
# Também podemos usar o timestamp com uma semana a frente.
pd.Timestamp('9/4/2016') + pd.offsets.Week()

In [ ]:
# Podemos ir para o final do mês
pd.Timestamp('9/4/2016') + pd.offsets.MonthEnd()

### Working with Dates in a Dataframe

In [ ]:
# Vamos supor que queremos para 9 medidas: 'domingo de 2 em 2 semanas' começando em Outubro/2016.
# Usando o 'date_range', podemos criar esse DatetimeIndex.
# No 'date_range', precisamos especificar a data de inicio ou a data final. Então especificar quantos períodos queremos e a frequência.
dates = pd.date_range('10-01-2016', periods=9, freq='2W-SUN')
dates

In [ ]:
# Existem varios jeitos de especificar uma frequência, por exemplo o dia útil 'business day'
pd.date_range('10-01-2016', periods=9, freq='B')

In [ ]:
# Ou 1 quarto do ano = 3 meses
pd.date_range('04-01-2016', periods=12, freq='QS-JUN')

In [ ]:
# Voltando ao 'dates'. Vamos criar um dataframe utilizando essas datas
dates = pd.date_range('10-01-2016', periods=9, freq='2W-SUN')
df = pd.DataFrame({'Count 1': 100 + np.random.randint(-5, 10, 9).cumsum(),
                  'Count 2': 120 + np.random.randint(-5, 10, 9)}, index=dates)
df

In [ ]:
# Primeiro, podemos checar qual dia da semana para verificar.
df.index.weekday

In [ ]:
# Podemos usar a função diff() para encontrar a diferença entre os valores: diff() calcula a diferença na célula atual com a mais acima
df.diff()

In [ ]:
# Vamos supor que queremos saber a média de cada mês. Podemos fazer isso utilizando o 'resample' que agrupa os indices pelo o que especificarmos.
df.resample('M').mean()

In [ ]:
# Podemos pegar as colunas que satisfazem um ano por exemplo:
df.loc['2017']

In [ ]:
# Ou pegar um mês em particular
df.loc['2016-12']

In [ ]:
# Podemos pegar uma determinada para frente
df.loc['2016-12':]